Importing the Necessary Libraries and the links required

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url,'lxml')

Processing the data using Beautifulsoup, and cleaning up the to achieve the first df output

In [2]:
My_table = soup.find('table',{'class':'wikitable sortable'})
A1= ""
A2= ""
A3= ""
i=0;
PostalCode=[]
temp_PostalCode=None
Borough=[]
temp_Borough=None
Neighbourhood=[]
temp_Neighbourhood=None
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        A1 = cells[0].find(text=True)
        temp_PostalCode=str(A1)
        
        A2 = cells[1].findAll(text=True)
        temp_Borough=str(A2).replace(r"['","")
        temp_Borough=temp_Borough.replace(r"']","")
        temp_Borough=temp_Borough.replace("[\"","")
        temp_Borough=temp_Borough.replace("\"]","")
                  
        A3 = cells[2].find(text=True)     
        temp_Neighbourhood=str(A3).replace("\n","")
           
        
        PostalCode.append(temp_PostalCode)
        Borough.append(temp_Borough)
        Neighbourhood.append(temp_Neighbourhood)
        i=i+1
        #print('{} {} {} {}'.format(A1,A2,A3,i))        
df=pd.DataFrame()
df['PostalCode']=PostalCode
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood

The functions used as an Aggregate function to combine the strings for duplicates and Neighbourhoodless boroughs

In [3]:
def custom_add(k):
    c=''
    for value in k:
        c=c+', '+value
    z=len(c)
    c=c[2:z]
    return c

def ignore_value(j):
    previous=''
    result=''
    for name in j:
        if(result.find(name)==-1):
            result=result+name
    return result

In [4]:
df.sort_values('PostalCode',inplace=True)
df=df.groupby('PostalCode').agg({'Borough':ignore_value,'Neighbourhood':custom_add},inplace=True).reset_index()
x,y=df.shape
for z in range(x):
    if (((df.iloc[z,1]!='Not assigned')) and (df.iloc[z,2]=='Not assigned')):
        df.iloc[z,2]=df.iloc[z,1]
k=(df['Borough']!="Not assigned")
df=df[k]
df.head(12)

,PostalCode,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
3,M1E,Scarborough,"West Hill, Morningside, Guildwood"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
8,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
10,M1N,Scarborough,"Cliffside West, Birch Cliff"


Shape of the dataframe

In [5]:
df.shape

(103, 3)

Read the data from the CSV, before using join to combine dataframe based on PostalCode

In [6]:
url="http://cocl.us/Geospatial_data"
LatLon=pd.read_csv(url)

LatLon.rename(columns={"Postal Code":"PostalCode"},inplace=True)
df.set_index("PostalCode",inplace=True)
LatLon.set_index("PostalCode",inplace=True)
C=df.join(LatLon)
C.reset_index().head(12)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


Shape of the resulting dataframe C

In [7]:
C.reset_index().shape

(103, 5)

In [8]:
# To obtain coordinates necessary in order to use Folium to plot map
address = 'Toronto, Ontario'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

C:\Anaconda3\envs\Test_env\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


Foursquare queries and the necessary function to find shops that are located at the respective Boroughs

In [9]:
#Foursquare Queries
CLIENT_ID = 'PGYPKJK5TVI4PZ1I1D2RTLUYE5E1KC3SBPWB5NEGNAXVX1B3' # your Foursquare ID
CLIENT_SECRET = 'KFEBOQ5SGVDCDA4223TNL2H00JQA3YDPAHEXSEFD0B1GJK5F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=C['Neighbourhood'],
                                   latitudes=C['Latitude'],
                                   longitudes=C['Longitude']
                                  )


Your credentails:
CLIENT_ID: PGYPKJK5TVI4PZ1I1D2RTLUYE5E1KC3SBPWB5NEGNAXVX1B3
CLIENT_SECRET:KFEBOQ5SGVDCDA4223TNL2H00JQA3YDPAHEXSEFD0B1GJK5F
Malvern, Rouge
Port Union, Rouge Hill, Highland Creek
West Hill, Morningside, Guildwood
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Oakridge, Golden Mile, Clairlea
Cliffcrest, Cliffside, Scarborough Village West
Cliffside West, Birch Cliff
Scarborough Town Centre, Dorset Park, Wexford Heights
Maryvale, Wexford
Agincourt
Sullivan, Clarks Corners, Tam O'Shanter
Milliken, L'Amoreaux East, Agincourt North, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Oriole, Henry Farm, Fairview
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Wilson Heights, Bathurst Manor, Downsview North
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central


In [10]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Agincourt----
                venue  freq
0      Sandwich Place   0.2
1      Breakfast Spot   0.2
2              Lounge   0.2
3        Skating Rink   0.2
4  Chinese Restaurant   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Gym  0.11
2     Coffee Shop  0.11
3             Pub  0.11
4  Sandwich Place  0.11


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.12
1  Fast Food Restaurant  0.08
2             Juice Bar  0.08
3           Coffee Shop  0.08
4         Grocery Store  0.04


--

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Gym,Hotel,Bakery,Bar
1,Agincourt,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Chinese Restaurant,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
2,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Gym,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Doner Restaurant,Dim Sum Restaurant
3,Bayview Village,Japanese Restaurant,Bank,Café,Chinese Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Fast Food Restaurant,Coffee Shop,Juice Bar,Greek Restaurant,Thai Restaurant,Sandwich Place,Liquor Store,Comfort Food Restaurant,Restaurant


To observe the characteristics of the store, a KMeans approach is taken, grouping the businesses in order to any possible patterns. 

In [12]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 4])

In [13]:
toronto_grouped.head()
toronto_clustered=pd.DataFrame(toronto_grouped['Neighbourhood'])

temp=pd.DataFrame(kmeans.labels_)
toronto_clustered['Grouping']=temp

toronto_clustered.set_index('Neighbourhood',inplace=True)
C=C.reset_index().set_index('Neighbourhood')
toronto_clustered=toronto_clustered.join(C)


toronto_clustered=toronto_clustered.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'))
toronto_clustered.reset_index(inplace=True)



In [14]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clustered['Latitude'], toronto_clustered['Longitude'], toronto_clustered['Neighbourhood'], toronto_clustered['Grouping']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

From the map, the majority of the clusters are clusters 1, which seemed to surround the geographical city center of the city. From the list of venues ranked based on their frequency, they contain a wide variety of service type businesses and restaurants, suggesting that the areas are most likely the main business districts, and the shops are catered to the needs of the working people, whom spent most of their time there. 

Grouping 4 consist mainly of parks, which makes them likely residential areas. The other smaller groupings are likely minority areas since there seems to contain niche places that seemed to cater to their specific communities respectively.

It is important to note that not all Boroughs have business listed as area of interest by Foursquare, since the dataframe that resulted is less than the dataframe containing only the postcodes. 



In [15]:
toronto_clustered.loc[toronto_clustered['Grouping'] == 0, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,Grouping,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,0,-79.532242,Baseball Field,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
48,0,-79.498509,Locksmith,Baseball Field,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant


In [16]:
toronto_clustered.loc[toronto_clustered['Grouping'] == 1, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,Grouping,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,-79.384568,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Gym,Hotel,Bakery,Bar
1,1,-79.262029,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Chinese Restaurant,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
2,1,-79.543484,Pizza Place,Coffee Shop,Skating Rink,Gym,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Doner Restaurant,Dim Sum Restaurant
3,1,-79.385975,Japanese Restaurant,Bank,Café,Chinese Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
4,1,-79.419750,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Juice Bar,Greek Restaurant,Thai Restaurant,Sandwich Place,Liquor Store,Comfort Food Restaurant,Restaurant
5,1,-79.373306,Coffee Shop,Cocktail Bar,Restaurant,Pub,Cheese Shop,Bakery,Café,Italian Restaurant,Steakhouse,Farmers Market
6,1,-79.577201,Park,Pharmacy,Beer Store,Pizza Place,Liquor Store,Convenience Store,Café,Yoga Studio,Dumpling Restaurant,Dog Run
7,1,-79.428191,Café,Breakfast Spot,Coffee Shop,Stadium,Furniture / Home Store,Burrito Place,Caribbean Restaurant,Climbing Gym,Bar,Italian Restaurant
8,1,-79.321558,Light Rail Station,Farmers Market,Garden Center,Smoke Shop,Fast Food Restaurant,Spa,Brewery,Burrito Place,Restaurant,Auto Workshop
11,1,-79.615819,Hotel,Coffee Shop,American Restaurant,Gym / Fitness Center,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,Middle Eastern Restaurant,Sandwich Place,Doner Restaurant


In [17]:
toronto_clustered.loc[toronto_clustered['Grouping'] == 2, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,Grouping,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,2,-79.383160,Restaurant,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
68,2,-79.239476,Jewelry Store,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [18]:
toronto_clustered.loc[toronto_clustered['Grouping'] == 3, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,Grouping,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,3,-79.374714,Martial Arts Dojo,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [19]:
toronto_clustered.loc[toronto_clustered['Grouping'] == 4, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

,Grouping,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,4,-79.464763,Bus Stop,Park,Airport,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
10,4,-79.453512,Park,Fast Food Restaurant,Women's Store,Pharmacy,Market,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
31,4,-79.338106,Metro Station,Park,Coffee Shop,Convenience Store,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
34,4,-79.411307,Park,Bus Line,Sushi Restaurant,Jewelry Store,Trail,Donut Shop,Discount Store,Dog Run,Doner Restaurant,Drugstore
35,4,-79.445073,Park,Japanese Restaurant,Pub,Playground,Bakery,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
38,4,-79.428191,Trail,Park,Field,Hockey Arena,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
41,4,-79.388790,Park,Bus Line,Swim School,Dim Sum Restaurant,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
47,4,-79.284577,Park,Coffee Shop,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
56,4,-79.506944,Park,River,Pool,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
58,4,-79.329656,Fast Food Restaurant,Park,Food & Drink Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
